# Обучение моделей классификации

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("dataset_clusterd.csv", sep="\t", encoding="utf-16")
df.drop(columns=['cluster_kmeans_full', 'cluster_ngram2'],inplace=True)
df.head()

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,count_description,count_title,count_titlefeat,cluster_kmeans,named_clusters
0,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,обработка поверхности,15.0,специалист должен знать понимать специализиров...,256,2,48,3,Информационные и коммуникационные технологии
1,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сложных элементов ювелирных издел...,20.0,специалист должен знать понимать способы испол...,256,9,38,3,Информационные и коммуникационные технологии
2,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление простых элементов ювелирных изделий,20.0,специалист должен знать понимать способы испол...,256,5,15,3,Информационные и коммуникационные технологии
3,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,подготовка драгоценных сплавов изготовления эл...,10.0,специалист должен знать понимать свойства спос...,256,7,21,3,Информационные и коммуникационные технологии
4,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сплавов драгоценных металлов,5.0,специалист должен знать понимать состав сплаво...,256,4,56,3,Информационные и коммуникационные технологии


Для того, чтобы можно было использовать все признаки, воспользуемся трансформаторами BaseEstimator, TransformerMixin и конвеером Pipeline. Также рандомизируем датасет для классификации, чтобы было более равномерное распределение данных на обучающей и тестовой выборке

In [3]:
df=df.sample(n=len(df))

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.key]
    
class NumericalSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.key]]

In [5]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian") 

def stemming_tokenizer(text):
    text = re.split('\W+', text)
    text = [stemmer.stem(word) for word in text]
    return text

In [6]:
from nltk.corpus import stopwords
import re
stop_words = stopwords.words("russian")
df.head()

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,count_description,count_title,count_titlefeat,cluster_kmeans,named_clusters
775,Преподавание в младших классах(юниоры),высококвалифицированные специалисты сфере нача...,взаимодействие родителями сотрудниками образов...,25.0,специалист должен знать понимать основные доку...,266,5,96,2,Производство и инженерные технологии
473,"Физическая культура, спорт и фитнес(юниоры)",компетенция физическая культура спорт фитнес в...,планирование организации содержания физкультур...,48.0,специалист должен знать понимать требования фо...,224,6,190,2,Производство и инженерные технологии
67,Фотография,документальная точность фотографических изобра...,технические аспекты общие характеристики,20.0,специалист должен знать понимать технологическ...,186,4,31,1,Сфера услуг
526,Технологии информационного моделирования BIM,компетенция тим актуальна архитектурно строите...,выполнение работ,15.0,специалист должен знать понимать проектное упр...,186,2,47,6,Строительство и строительные технологии
302,Лабораторный медицинский анализ (Юниоры),сегодня лабораторная диагностика это широкий с...,проведение лабораторных биохимических исследов...,18.0,специалист должен знать понимать задачи структ...,144,4,62,5,Образование


## Обучение
Для обучения необходимы пробразованные признаки для классификации. Построим конвейер для дальнейшего обучения моделей.

In [7]:
dictionary = Pipeline([
    ('selector', TextSelector(key="описание_компетенции_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer))
    ])

dictionary.fit_transform(df).todense()

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'впроч', 'всег', 'всегд', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс', 'теб', 'тепер', 'тог', 'тогд', 'тож', 'тольк', 'хорош', 'хот', 'чег', 'чут', 'эт'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.06345781, 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [8]:
title = Pipeline([
    ('selector', TextSelector(key="название_раздела_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer))
    ])

title.fit_transform(df).todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
ves_title = Pipeline([
    ('selector', NumericalSelector(key="значимость_раздела_wsss")),
    ('tfidf', StandardScaler())
    ])

ves_title.fit_transform(df)

array([[ 0.868174  ],
       [ 2.98332836],
       [ 0.40835783],
       [-0.05145834],
       [ 0.22443136],
       [ 0.0405049 ],
       [ 1.32799016],
       [ 4.36277686],
       [-0.5112745 ],
       [-0.05145834],
       [ 0.5922843 ],
       [-0.97109067],
       [-0.5112745 ],
       [-0.5112745 ],
       [-0.69520097],
       [-0.97109067],
       [ 0.40835783],
       [ 0.13246813],
       [-0.05145834],
       [-0.5112745 ],
       [ 0.68424753],
       [ 1.32799016],
       [ 0.40835783],
       [ 0.68424753],
       [-0.5112745 ],
       [-0.97109067],
       [ 0.40835783],
       [-0.5112745 ],
       [-0.32734803],
       [-0.2353848 ],
       [ 0.68424753],
       [-0.5112745 ],
       [-0.5112745 ],
       [-0.32734803],
       [-0.97109067],
       [ 0.40835783],
       [-0.32734803],
       [-0.97109067],
       [-0.14342157],
       [ 0.96013723],
       [-0.05145834],
       [ 0.0405049 ],
       [ 2.2476225 ],
       [-0.97109067],
       [-0.60323773],
       [-0

In [10]:
title_feat = Pipeline([
    ('selector', TextSelector(key="признаки_раздела_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer))
    ])

title_feat.fit_transform(df).todense()

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'впроч', 'всег', 'всегд', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс', 'теб', 'тепер', 'тог', 'тогд', 'тож', 'тольк', 'хорош', 'хот', 'чег', 'чут', 'эт'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
count_desc = Pipeline([
    ('selector', NumericalSelector(key="count_description")),
    ('tfidf', StandardScaler())
    ])

count_desc.fit_transform(df)

array([[ 0.48579607],
       [ 0.04849435],
       [-0.34715959],
       [-0.34715959],
       [-0.78446131],
       [ 1.62069816],
       [-0.53457462],
       [-0.31592375],
       [-0.3263357 ],
       [-0.45127905],
       [-0.01397733],
       [-0.95105245],
       [-0.95105245],
       [-0.34715959],
       [-0.35757154],
       [-0.5762224 ],
       [-0.5762224 ],
       [-0.78446131],
       [-0.5762224 ],
       [ 0.91268585],
       [-1.08640774],
       [-0.40963127],
       [-0.3263357 ],
       [-1.47164974],
       [-0.26386403],
       [ 0.63156331],
       [ 0.9022739 ],
       [-0.63869407],
       [ 1.74564151],
       [-0.87816883],
       [-0.87816883],
       [ 0.04849435],
       [ 3.65102759],
       [-0.14933262],
       [-0.18056846],
       [-1.04475996],
       [ 1.56863843],
       [-0.20139235],
       [-0.20139235],
       [-1.18011525],
       [-1.63824087],
       [ 3.2449617 ],
       [ 0.48579607],
       [-0.10768484],
       [-1.54453336],
       [-0

In [12]:
count_titl = Pipeline([
    ('selector', NumericalSelector(key="count_title")),
    ('tfidf', StandardScaler())
    ])

count_desc.fit_transform(df)

array([[ 0.48579607],
       [ 0.04849435],
       [-0.34715959],
       [-0.34715959],
       [-0.78446131],
       [ 1.62069816],
       [-0.53457462],
       [-0.31592375],
       [-0.3263357 ],
       [-0.45127905],
       [-0.01397733],
       [-0.95105245],
       [-0.95105245],
       [-0.34715959],
       [-0.35757154],
       [-0.5762224 ],
       [-0.5762224 ],
       [-0.78446131],
       [-0.5762224 ],
       [ 0.91268585],
       [-1.08640774],
       [-0.40963127],
       [-0.3263357 ],
       [-1.47164974],
       [-0.26386403],
       [ 0.63156331],
       [ 0.9022739 ],
       [-0.63869407],
       [ 1.74564151],
       [-0.87816883],
       [-0.87816883],
       [ 0.04849435],
       [ 3.65102759],
       [-0.14933262],
       [-0.18056846],
       [-1.04475996],
       [ 1.56863843],
       [-0.20139235],
       [-0.20139235],
       [-1.18011525],
       [-1.63824087],
       [ 3.2449617 ],
       [ 0.48579607],
       [-0.10768484],
       [-1.54453336],
       [-0

In [13]:
count_titfe = Pipeline([
    ('selector', NumericalSelector(key="count_titlefeat")),
    ('tfidf', StandardScaler())
    ])

count_desc.fit_transform(df)

array([[ 0.48579607],
       [ 0.04849435],
       [-0.34715959],
       [-0.34715959],
       [-0.78446131],
       [ 1.62069816],
       [-0.53457462],
       [-0.31592375],
       [-0.3263357 ],
       [-0.45127905],
       [-0.01397733],
       [-0.95105245],
       [-0.95105245],
       [-0.34715959],
       [-0.35757154],
       [-0.5762224 ],
       [-0.5762224 ],
       [-0.78446131],
       [-0.5762224 ],
       [ 0.91268585],
       [-1.08640774],
       [-0.40963127],
       [-0.3263357 ],
       [-1.47164974],
       [-0.26386403],
       [ 0.63156331],
       [ 0.9022739 ],
       [-0.63869407],
       [ 1.74564151],
       [-0.87816883],
       [-0.87816883],
       [ 0.04849435],
       [ 3.65102759],
       [-0.14933262],
       [-0.18056846],
       [-1.04475996],
       [ 1.56863843],
       [-0.20139235],
       [-0.20139235],
       [-1.18011525],
       [-1.63824087],
       [ 3.2449617 ],
       [ 0.48579607],
       [-0.10768484],
       [-1.54453336],
       [-0

In [14]:
df.head()

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,count_description,count_title,count_titlefeat,cluster_kmeans,named_clusters
775,Преподавание в младших классах(юниоры),высококвалифицированные специалисты сфере нача...,взаимодействие родителями сотрудниками образов...,25.0,специалист должен знать понимать основные доку...,266,5,96,2,Производство и инженерные технологии
473,"Физическая культура, спорт и фитнес(юниоры)",компетенция физическая культура спорт фитнес в...,планирование организации содержания физкультур...,48.0,специалист должен знать понимать требования фо...,224,6,190,2,Производство и инженерные технологии
67,Фотография,документальная точность фотографических изобра...,технические аспекты общие характеристики,20.0,специалист должен знать понимать технологическ...,186,4,31,1,Сфера услуг
526,Технологии информационного моделирования BIM,компетенция тим актуальна архитектурно строите...,выполнение работ,15.0,специалист должен знать понимать проектное упр...,186,2,47,6,Строительство и строительные технологии
302,Лабораторный медицинский анализ (Юниоры),сегодня лабораторная диагностика это широкий с...,проведение лабораторных биохимических исследов...,18.0,специалист должен знать понимать задачи структ...,144,4,62,5,Образование


### Сборка всех признаков
Введём в FeatureUnion (метод, объединяющий признаки) пробразованные признаки, которых достаточно для модели классификации. 

In [15]:
from sklearn.pipeline import FeatureUnion
feat = FeatureUnion([("title", title),
                    ("ves_title", ves_title),
                    ("title_feat", title_feat)])

X=feat.fit_transform(df).todense()

### Разделение набора данных

In [16]:
from sklearn.model_selection import train_test_split
y=df["named_clusters"]
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1)

## Методы классификации
### SVC (Метод опорных векторов)
Метод Опорных Векторов или SVM (от англ. Support Vector Machines) — это линейный алгоритм используемый в задачах классификации и регрессии. Данный алгоритм имеет широкое применение на практике и может решать как линейные так и нелинейные задачи.
### Naive Bayes
Такой классификатор вычисляет вероятность принадлежности объекта к какому-то классу. Эта вероятность вычисляется из шанса, что какое-то событие произойдёт, с опорой на уже на произошедшие события. Каждый параметр классифицируемого объекта считается независимым от других параметров.
### Logistic Regression
Логистическая регрессия выводит прогнозы о точках в бинарном масштабе — нулевом или единичном. Если значение чего-либо равно либо больше 0.5, то объект классифицируется в большую сторону (к единице). Если значение меньше 0.5 — в меньшую (к нулю).

In [17]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

### Метрики 
В качестве метрик возьмем все то, что располагает в себе метод `classification_report`. Берутся эти метрики потому что они предоставлют достаточно информации, чтобы сравнить несколько моделей классификации и выбрать по этим метрикам самую лучшую модель. А именно:
* `accuracy` - доля правильных ответов алгоритма
* `precision` - можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными
* `recall` - показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм
* `f-score` - эта мера может быть интерпретирована как средневзвешенное значение Precision и recall, где оценка F1 достигает своего наилучшего значения при 1 и худшего значения при 0
* `macro avg` - вычисляет метрику независимо для каждого класса, а затем принимает среднее значение
* `weighted avg` - вычисляет оценку F1 для каждого класса независимо, но когда она складывает их вместе, используется вес, который зависит от количества истинных меток каждого класса

Атрибут `support` - показывает количество образцов истинного ответа, которые лежат в этом классе.

In [18]:
from sklearn.metrics import classification_report

## Обучение моделей и их сравнение

In [19]:
pipeline_svc=SVC()

pipeline_svc.fit(X_train,y_train)
preds_scv=pipeline_svc.predict(X_test)
print(classification_report(y_test, preds_scv))

                                              precision    recall  f1-score   support

Информационные и коммуникационные технологии       0.86      0.86      0.86        21
                                 Образование       0.91      0.91      0.91        11
        Производство и инженерные технологии       0.00      0.00      0.00         1
     Строительство и строительные технологии       0.71      0.83      0.77        18
                                 Сфера услуг       0.78      0.74      0.76        19
                         Творчество и дизайн       1.00      0.90      0.95        10
                       Транспорт и логистика       1.00      1.00      1.00         1

                                    accuracy                           0.83        81
                                   macro avg       0.75      0.75      0.75        81
                                weighted avg       0.82      0.83      0.82        81



/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Метод `SVC` показал accuracy: **` `**.

In [20]:
pipeline_gnb=GaussianNB()

pipeline_gnb.fit(X_train,y_train)
preds_gnb=pipeline_gnb.predict(X_test)
print(classification_report(y_test, preds_gnb))

                                              precision    recall  f1-score   support

Информационные и коммуникационные технологии       0.86      0.90      0.88        21
                                 Образование       0.85      1.00      0.92        11
        Производство и инженерные технологии       1.00      1.00      1.00         1
     Строительство и строительные технологии       0.85      0.94      0.89        18
                                 Сфера услуг       0.93      0.68      0.79        19
                         Творчество и дизайн       1.00      1.00      1.00        10
                       Транспорт и логистика       1.00      1.00      1.00         1

                                    accuracy                           0.89        81
                                   macro avg       0.93      0.93      0.93        81
                                weighted avg       0.89      0.89      0.89        81



Метод `GaussianNB` показал accuracy: **` `**.

In [21]:
pipeline_lr=LogisticRegression()

pipeline_lr.fit(X_train,y_train)
preds_lr=pipeline_lr.predict(X_test)
print(classification_report(y_test, preds_lr))

                                              precision    recall  f1-score   support

Информационные и коммуникационные технологии       0.90      0.86      0.88        21
                                 Образование       0.79      1.00      0.88        11
        Производство и инженерные технологии       0.00      0.00      0.00         1
     Строительство и строительные технологии       0.80      0.89      0.84        18
                                 Сфера услуг       0.82      0.74      0.78        19
                         Творчество и дизайн       1.00      0.90      0.95        10
                       Транспорт и логистика       1.00      1.00      1.00         1

                                    accuracy                           0.85        81
                                   macro avg       0.76      0.77      0.76        81
                                weighted avg       0.85      0.85      0.85        81



/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Метод `LogisticRegression` показал accuracy: **` `**.

**Вывод по обучению:** как видим из представленных данных лучше всего себя показал классификатор с точностью **` `**.

# Feature Engineering

### Сравнение моделей
Обучать будем на классификаторе `LogisticRegression`, т.к. он показал лучше всего себя. Необходимо сравнить точность прошлого классификатора с 3 признаками и того, который будет представлен в **`Feature Engineering`** со всеми возможними признаками

### Преобразование данных
Как уже было ранее сказано, будет призведенно преобразование данных от текстового формата к числовому, путём подсчёта каждой буквы текстового признака

In [22]:
df["count_description_len"]=np.nan
df["count_title_len"]=np.nan
df["count_titlefeat_len"]=np.nan

for row in range(len(df)):
    df["count_description_len"][row]=len(df["описание_компетенции_wsss"][row]) 
    
for row in range(len(df)):
    df["count_title_len"][row]=len(df["название_раздела_wsss"][row]) 
    
for row in range(len(df)):
    df["count_titlefeat_len"][row]=len(df["признаки_раздела_wsss"][row]) 
    
df["count_description_len"]=df["count_description_len"].astype(int)
df["count_title_len"]=df["count_title_len"].astype(int)
df["count_titlefeat_len"]=df["count_titlefeat_len"].astype(int)
df.head()

<ipython-input-22-552de664d941>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["count_description_len"][row]=len(df["описание_компетенции_wsss"][row])
<ipython-input-22-552de664d941>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["count_title_len"][row]=len(df["название_раздела_wsss"][row])
<ipython-input-22-552de664d941>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["count_titlefeat_len"][row]=len(df["признаки_раздела

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,count_description,count_title,count_titlefeat,cluster_kmeans,named_clusters,count_description_len,count_title_len,count_titlefeat_len
775,Преподавание в младших классах(юниоры),высококвалифицированные специалисты сфере нача...,взаимодействие родителями сотрудниками образов...,25.0,специалист должен знать понимать основные доку...,266,5,96,2,Производство и инженерные технологии,2760,66,986
473,"Физическая культура, спорт и фитнес(юниоры)",компетенция физическая культура спорт фитнес в...,планирование организации содержания физкультур...,48.0,специалист должен знать понимать требования фо...,224,6,190,2,Производство и инженерные технологии,2191,72,1978
67,Фотография,документальная точность фотографических изобра...,технические аспекты общие характеристики,20.0,специалист должен знать понимать технологическ...,186,4,31,1,Сфера услуг,1778,40,315
526,Технологии информационного моделирования BIM,компетенция тим актуальна архитектурно строите...,выполнение работ,15.0,специалист должен знать понимать проектное упр...,186,2,47,6,Строительство и строительные технологии,1867,16,481
302,Лабораторный медицинский анализ (Юниоры),сегодня лабораторная диагностика это широкий с...,проведение лабораторных биохимических исследов...,18.0,специалист должен знать понимать задачи структ...,144,4,62,5,Образование,1583,50,601


### Подгонка преобразованных данных

In [23]:
count_desc_len = Pipeline([
    ('selector', NumericalSelector(key="count_description_len")),
    ('tfidf', StandardScaler())
    ])

count_desc_len.fit_transform(df)

array([[ 0.68857593],
       [ 0.06384189],
       [-0.38961181],
       [-0.29189419],
       [-0.60371223],
       [ 1.87985261],
       [-0.42694214],
       [-0.28420853],
       [-0.33032247],
       [-0.55650034],
       [-0.06901016],
       [-1.10876962],
       [-1.10876962],
       [-0.29189419],
       [-0.48183968],
       [-0.54552083],
       [-0.54552083],
       [-0.60371223],
       [-0.46756632],
       [ 0.88181528],
       [-1.00226839],
       [-0.47305608],
       [-0.33032247],
       [-1.55124382],
       [-0.2490741 ],
       [ 0.54474437],
       [ 1.11458086],
       [-0.4434114 ],
       [ 1.80738785],
       [-1.03850077],
       [-1.03850077],
       [ 0.06384189],
       [ 3.85836005],
       [-0.11622205],
       [-0.03058188],
       [-1.13731634],
       [ 1.44286817],
       [-0.25346591],
       [-0.17660935],
       [-1.03850077],
       [-1.55453767],
       [ 3.04697437],
       [ 0.68857593],
       [-0.03387573],
       [-1.61492497],
       [-0

In [24]:
count_titl_len = Pipeline([
    ('selector', NumericalSelector(key="count_title_len")),
    ('tfidf', StandardScaler())
    ])

count_titl_len.fit_transform(df)

array([[ 2.09757954e-01],
       [ 2.73696779e-01],
       [-6.73102895e-02],
       [-3.23065591e-01],
       [ 3.92544195e-02],
       [ 5.50765022e-01],
       [-2.16500882e-01],
       [-3.55035004e-01],
       [-1.73874998e-01],
       [-1.95187940e-01],
       [-1.20592644e-01],
       [-4.59973477e-02],
       [-2.05844411e-01],
       [-3.65691474e-01],
       [ 1.24506187e-01],
       [-2.05844411e-01],
       [ 1.03193245e-01],
       [ 2.73696779e-01],
       [-8.86232313e-02],
       [-2.48470295e-01],
       [-1.41905586e-01],
       [-5.66538186e-02],
       [-2.48470295e-01],
       [-1.41905586e-01],
       [ 2.85979486e-02],
       [-1.09936173e-01],
       [-1.31249115e-01],
       [-1.73874998e-01],
       [ 7.85207382e-01],
       [-3.53408768e-02],
       [-1.09936173e-01],
       [ 9.45054445e-01],
       [-2.69783236e-01],
       [-1.73874998e-01],
       [-8.86232313e-02],
       [-8.86232313e-02],
       [-3.65691474e-01],
       [-3.53408768e-02],
       [-3.2

In [25]:
count_titlfe_len = Pipeline([
    ('selector', NumericalSelector(key="count_titlefeat_len")),
    ('tfidf', StandardScaler())
    ])

count_titlfe_len.fit_transform(df)

array([[ 4.62102213e-01],
       [ 2.41394853e+00],
       [-8.58148673e-01],
       [-5.31529229e-01],
       [-2.95418787e-01],
       [ 2.92330701e-02],
       [ 7.17888525e-01],
       [ 1.27078048e+00],
       [-1.59655283e-01],
       [-1.39529993e+00],
       [-7.00741712e-01],
       [-1.38011826e-01],
       [-3.91830551e-01],
       [-1.01752322e+00],
       [-8.22732107e-01],
       [ 1.71545514e+00],
       [-2.71807743e-01],
       [-3.11159483e-01],
       [ 6.85848104e-02],
       [-8.16829346e-01],
       [ 6.45087805e-01],
       [ 1.20388252e+00],
       [-7.45996213e-01],
       [-7.11138676e-02],
       [-8.77824543e-01],
       [-4.98080250e-01],
       [ 1.61314062e+00],
       [ 8.94971356e-01],
       [-1.07458324e+00],
       [-7.24352756e-01],
       [ 3.18468361e-01],
       [ 1.13304938e+00],
       [ 8.71360312e-01],
       [ 2.72654831e-02],
       [ 6.80504372e-01],
       [-9.30949393e-01],
       [ 1.17240113e+00],
       [-7.08612060e-01],
       [-1.2

### Формирование конвейера со всеми признаками

In [26]:
from sklearn.pipeline import FeatureUnion
feat = FeatureUnion([("title", title),
                    ("ves_title", ves_title),
                    ("title_feat", title_feat),
                    ("count_desc", count_desc),
                    ("count_titl", count_titl),
                    ("count_titfe", count_titfe),
                    ("count_desc_len", count_desc_len),
                    ("count_titl_len", count_titl_len),
                    ("count_titlfe_len", count_titlfe_len)])

X=feat.fit_transform(df).todense()

In [27]:
y=df["named_clusters"]
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1)

In [28]:
pipeline_lr_f=LogisticRegression()

pipeline_lr_f.fit(X_train,y_train)
preds_lr_f=pipeline_lr_f.predict(X_test)
print(classification_report(y_test, preds_lr_f))

                                              precision    recall  f1-score   support

Информационные и коммуникационные технологии       0.93      1.00      0.97        14
                                 Образование       0.91      0.77      0.83        13
        Производство и инженерные технологии       1.00      0.86      0.92         7
     Строительство и строительные технологии       0.95      0.86      0.90        22
                                 Сфера услуг       0.58      0.88      0.70         8
                         Творчество и дизайн       1.00      1.00      1.00        13
                       Транспорт и логистика       1.00      1.00      1.00         4

                                    accuracy                           0.90        81
                                   macro avg       0.91      0.91      0.90        81
                                weighted avg       0.92      0.90      0.91        81



Вывод по обучению: как видим из представленных данных классификатор **`Feature Engineering`** `LogisticRegression` показал себя чуть лучше по сравнению с прошлой моделью с тремя признаками, у которой точность была **` `**.

Точность **`Feature Engineering`** `LogisticRegression` составляет **` `**.

# Подготовка отчёта:
* **Показатели точности работы модели:** Во внимание можно взять абсолютно все метрики, поставляющиеся в методе `classification_report`, но за основу сравнения моделей была взята метрика accuracy, которая показывает долю правильных ответов алгоритма
* **Точность выбранной модели в сравнении с другими:** Все модели показали достаточно хороший результат на отметке не ниже **`0.78`**, но модель `LogisticRegression` показала себя лучше всего, на отметке **` `**. Считаю такой результат получился, потому что этот классификатор достаточно хорошо работает с большим количеством признаков и не требует больших вычислительных мощностей из-за составления линейного алгоритма
* **Результаты работы `Feature Engineering`:** Благодаря тому, что линейный классификатор способен работать с большим количеством признаков, то благодаря этому, результат точности модели улучшился. Использование дополнительных признаков способствовало получению результата классификатора **`Feature Engineering`** `LogisticRegression` составляет **` `**.